In [1]:
using Pkg
Pkg.activate(".")
Pkg.add("Distributions")
Pkg.instantiate()
Pkg.precompile()

  Activating project at `c:\Users\sarah\OneDrive\Documents\GitHub\BEE-4750-Final-Project`
   Resolving package versions...
      Compat entries added for 
  No Changes to `C:\Users\sarah\OneDrive\Documents\GitHub\BEE-4750-Final-Project\Project.toml`
  No Changes to `C:\Users\sarah\OneDrive\Documents\GitHub\BEE-4750-Final-Project\Manifest.toml`
  0 dependencies successfully precompiled in 23 seconds. 79 already precompiled.

The following 1 direct dependency failed to precompile:

BEE4750FinalProject 

Error: Missing source file for Base.PkgId(Base.UUID("12345678-1234-1234-1234-123456789012"), "BEE4750FinalProject"  0 dependencies successfully precompiled in 2 seconds. 79 already precompiled.

The following 1 direct dependency failed to precompile:

BEE4750FinalProject 

Error: Missing source file for Base.PkgId(Base.UUID("12345678-1234-1234-1234-123456789012"), "BEE4750FinalProject"  0 dependencies successfully precompiled in 2 seconds. 79 already precompiled.



Base.Precompilation.PkgPrecompileError: The following 1 direct dependency failed to precompile:

BEE4750FinalProject 

Error: Missing source file for Base.PkgId(Base.UUID("12345678-1234-1234-1234-123456789012"), "BEE4750FinalProject"

In [ ]:
using JuMP
using HiGHS
using Random

# Data

T = 1:30                                # time step (days/mo)

Q_data = rand(328767.12:541666.67, 30)    # Data center waste heat [MW] at each t
D_th = rand(13698.63:22260.27, 30)      # Greenhouse demand [MW] at each t
alpha = 1.5                             # Water evaporation factor [L/MWh rejected]

# Greenhouse Daily Thermal Demand Function (30 days)
function monthly_heating_demand_MWh(U::Float64, A::Float64, volume::Float64,
                                    ACH::Float64, Tin::Float64,
                                    Tout_avg::Vector{Float64})
    # Constants
    ρ = 1.2       # kg/m³, air density
    cp = 1005.0   # J/(kg·K), specific heat

    # Compute effective heat-loss coefficient (UA_eff)
    Vdot = ACH * volume / 3600.0      # m³/s
    UA_eff = U * A + ρ * cp * Vdot   # W/°C

    daily_MWh = Float64[]  # output array

    for Tout in Tout_avg
        ΔT = max(Tin - Tout, 0.0)       # Only heat if indoor > outdoor
        Q_W = UA_eff * ΔT                # W
        E_MWh = Q_W * 24.0 / 1e6         # MWh/day
        push!(daily_MWh, E_MWh)
    end

    return daily_MWh
end

# Greenhouse parameters
U = 3.5          # W/m²·K
A = 200.0        # m²
volume = 400.0   # m³
ACH = 1.0        # air changes per hour
Tin = 18.0       # °C, indoor setpoint

# Example: average daily outdoor temps for a month (°C)
Tout_avg_month = [5.0, 4.0, 6.0, 7.5, 8.0, 3.0, 2.5, 5.0, 6.0, 4.5,
                  5.0, 6.5, 7.0, 4.0, 3.5, 5.0, 6.0, 5.5, 4.0, 3.0,
                  2.5, 4.0, 5.5, 6.0, 4.5, 3.0, 5.0, 6.0, 4.5, 3.5]

# Compute daily heating demand
daily_demand = monthly_heating_demand_MWh(U, A, volume, ACH, Tin, Tout_avg_month)

# Model

model = Model(HiGHS.Optimizer)

@variable(model, x_reuse[t in T] >= 0)              # Heat reused in CEA
@variable(model, x_reject[t in T] >= 0)             # heat rejected to cooling tower

# Objective
@objective(model, Min, sum(alpha * x_reject[t] for t in T))

# Constraints
@constraint(model, [t in T], x_reuse[t] + x_reject[t] == Q_data[t])
@constraint(model, [t in T], x_reuse[t] <= D_th[t])

optimize!(model)

println("Optimal total water use [L]: ", objective_value(model))
println("\nHour  x_reuse[kW]  x_reject[kW]")

for t in T
    println(rpad(t, 5), " ",
            rpad(value(x_reuse[t]), 12), " ",
            rpad(value(x_reject[t]), 12))
end


Day 1: Heating demand = 0.26 MWh
Day 2: Heating demand = 0.28 MWh
Day 3: Heating demand = 0.24 MWh
Day 4: Heating demand = 0.21 MWh
Day 5: Heating demand = 0.2 MWh
Day 6: Heating demand = 0.3 MWh
Day 7: Heating demand = 0.31 MWh
Day 8: Heating demand = 0.26 MWh
Day 9: Heating demand = 0.24 MWh
Day 10: Heating demand = 0.27 MWh
Day 11: Heating demand = 0.26 MWh
Day 12: Heating demand = 0.23 MWh
Day 13: Heating demand = 0.22 MWh
Day 14: Heating demand = 0.28 MWh
Day 15: Heating demand = 0.29 MWh
Day 16: Heating demand = 0.26 MWh
Day 17: Heating demand = 0.24 MWh
Day 18: Heating demand = 0.25 MWh
Day 19: Heating demand = 0.28 MWh
Day 20: Heating demand = 0.3 MWh
Day 21: Heating demand = 0.31 MWh
Day 22: Heating demand = 0.28 MWh
Day 23: Heating demand = 0.25 MWh
Day 24: Heating demand = 0.24 MWh
Day 25: Heating demand = 0.27 MWh
Day 26: Heating demand = 0.3 MWh
Day 27: Heating demand = 0.26 MWh
Day 28: Heating demand = 0.24 MWh
Day 29: Heating demand = 0.27 MWh
Day 30: Heating demand = 0.